# ReAct Agent + RAG Tool

In this notebook we will build a [ReAct](https://react-lm.github.io/) agent capable of answering questions about specific source information/documents. This agent will use a technique known as [Retrieval Augmented Generation (RAG)](https://python.langchain.com/docs/concepts/rag/).

Retrieval Augmented Generation (RAG) is a powerful technique that enhances language models by combining them with external knowledge bases. RAG addresses a key limitation of models: models rely on fixed training datasets, which can lead to outdated or incomplete information.

When given a query:
1. RAG systems first search a knowledge base for relevant information
2. The system then incorporates this retrieved information into the model's prompt
3. The model uses the provided context to generate a response to the query.

By bridging the gap between vast language models and dynamic, targeted information retrieval, RAG is a powerful technique for building more capable and reliable AI systems.

A typical RAG application has two main components:
1. **Indexing**: a pipeline for ingesting data from a source and indexing it. *This usually happens offline.*
2. **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

In [1]:
# load the environment variables
import os
from dotenv import load_dotenv
load_dotenv(verbose=True)

True

## 1. Indexing
The indexing process follow these steps:
1. Load: First we need to load our data. We will use [LangChain's DirectoryLoader](https://python.langchain.com/docs/how_to/document_loader_directory/), a simple interface that allows us to load a range of file types out-of-the-box.
2. Split: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window. We will use [LangChain's RecursiveCharacterTextSplitter](https://python.langchain.com/docs/concepts/text_splitters/#text-structured-based) to split the documents.
3. Embed and Store: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a VectorStore and Embeddings model. In this notebook, we will use [Chroma](https://github.com/chroma-core/chroma), a simple and easy-to-use open-source embedding database.

![indexing-steps](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

The complete implementations of this process can be found in `src/hackathon/index_data.py`

### Load Documents

In this example, we'll load and ingest [Meta's Terms of Service](https://mbasic.facebook.com/legal/terms/plain_text_terms/) so that we can ask questions and better understand a document most of us have probably agreed to but never actually read!

In [2]:
from langchain_community.document_loaders import DirectoryLoader

data_directory = "../data/"

loader = DirectoryLoader(path=data_directory, show_progress=True, use_multithreading=True)

documents = loader.load()

print(f"Loaded {len(documents)} documents")

  0%|          | 0/1 [00:00<?, ?it/s]/Users/pedro/code/_general/intro-to-ai-agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1/1 [00:06<00:00,  6.15s/it]

Loaded 1 documents


### Split Documents

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_splits = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(doc_splits)} chunks")

Split 1 documents into 45 chunks


In [4]:
doc_splits

[Document(metadata={'source': '../data/Meta Terms of Service.pdf'}, page_content='Terms of Service\n\nExplore the policy\n\nOverview\n\n1. The services we provide\n\n2. How our services are funded\n\n3. Your commitments to Facebook and our community\n\n4. Additional provisions\n\n5. Other terms and policies that may apply to you\n\nOverview\n\nEffective January 1, 2025'),
 Document(metadata={'source': '../data/Meta Terms of Service.pdf'}, page_content='2. How our services are funded\n\n3. Your commitments to Facebook and our community\n\n4. Additional provisions\n\n5. Other terms and policies that may apply to you\n\nOverview\n\nEffective January 1, 2025\n\nMeta builds technologies and services that enable people to connect with each oth‐ er, build communities, and grow businesses. These Terms of Service (the "Terms") govern your access and use of Facebook, Messenger, and the other products, web‐ sites, features, apps, services, technologies, and software we offer (the Meta Products or

### Embed and Store Documents

In [5]:
from hackathon.config import embedding_model
from langchain_chroma import Chroma

vector_store_directory = "../vector_store/"

vector_store = Chroma(
    collection_name="meta_terms_of_service",
    embedding_function=embedding_model,
    persist_directory=vector_store_directory,
)

In [6]:
# Index chunks
chunk_indexes = vector_store.add_documents(documents=doc_splits)

## 2. Retrieval and generation
1. Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/docs/concepts/retrievers/).
2. Generate: A ChatModel / LLM produces an answer using a prompt that includes both the question with the retrieved data

![retrieval-and-generation](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

To do this, we will use the ReAct agent associated with a RAG tool.

In [7]:
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=3)
    serialized = (
        (
            "Use the following pieces of retrieved context to answer the question. "
            "If you don't know the answer, say that you don't know. "
            "Use three sentences maximum and keep the answer concise.\n\n"
        ) + "\n\n".join(
            (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
            for doc in retrieved_docs
        )
    )
    return serialized, retrieved_docs

In [8]:
from hackathon.agents.react.agent import ReActAgent
from hackathon.config import llm

react_rag_agent = ReActAgent(
    llm=llm,
    tools=[retrieve],
    system_prompt="You are a helpful assistant for question-answering tasks.",
)

### Run ReAct Agent with RAG tool

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

# Define the input
messages = [
    HumanMessage(content="What can Meta do with my personal data?"),
]

# Run the graph
react_output = react_rag_agent.run(input={"messages": messages})

In [10]:
# get messages and tool outputs
for m in react_output["messages"]:
    m.pretty_print()
    if isinstance(m, ToolMessage):
        print()
        print(f" --> Tool artifact: {m.artifact} (type: {type(m.artifact)})")

================================ Human Message =================================

What can Meta do with my personal data?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_JR9bqXDg43adcaYP9nptCF3A)
 Call ID: call_JR9bqXDg43adcaYP9nptCF3A
  Args:
    query: What can Meta (Facebook) do with my personal data?
================================= Tool Message =================================
Name: retrieve

Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.

Source: {'source': '../data/Meta Terms of Service.pdf'}
Content: access to certain features, disabling an account, or contacting law enforcement. We share data across Meta Companies when we detect misuse or harmful conduct by someone using one of our Products or to help keep Meta Products, users and the community safe. For example, we share informati

In [11]:
final_message = react_output["messages"][-1]
print(final_message.content)

Meta can access, preserve, use, and share your personal data when it believes it is required or permitted by law, or to keep its products, users, and the community safe. It shares information across Meta Companies, for example, when preventing misuse or promoting safety and compliance with the law. For more details, you should review Meta's Privacy Policy.
